# Example 4: Single Path PNE Analysis

This notebook demonstrates analyzing a single PNE folder:
- **Path**: `A1_MP1_4500mAh_T23_3`
- **Analysis Type**: Single folder analysis with profile extraction
- **Features**: Cycle data analysis, profile analysis at key cycles, statistical analysis

## 1. Setup and Imports

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Add parent directory to path for imports
sys.path.insert(0, os.path.dirname(os.path.abspath('__file__')))

# Import from BatteryDataTool
from BatteryDataTool import (
    extract_text_in_brackets, check_cycler,
    pne_min_cap, pne_search_cycle,
    pne_cycle_data,
    pne_chg_Profile_data, pne_dchg_Profile_data,
    pne_dcir_Profile_data,
    graph_cycle, graph_profile, graph_soc_dcir
)

print("✅ Import successful")

## 2. Configuration

In [ ]:
# Data path configuration
BASE_PATH = r"C:\Users\Ryu\Python_project\data\BatteryData_251010\Rawdata"
FOLDER_NAME = "A1_MP1_4500mAh_T23_3"

folder_path = os.path.join(BASE_PATH, FOLDER_NAME)

# Analysis parameters
ini_crate = 0.2  # Initial C-rate for capacity measurement
chkir = False     # Check internal resistance
chkir2 = False    # Check internal resistance (secondary)
mkdcir = True     # Make DCIR data

# Profile analysis parameters
cutoff = 0.0      # Cutoff voltage
smoothdegree = 51 # Smoothing degree for dQ/dV

print(f"📁 Data folder: {FOLDER_NAME}")
print(f"📂 Full path: {folder_path}")
print(f"✅ Folder exists: {os.path.exists(folder_path)}")

## 3. Verify Cycler Type and Extract Capacity

In [ ]:
# Check cycler type
cycler_type = check_cycler(folder_path)
print(f"🔍 Detected cycler type: {cycler_type}")

if cycler_type != 'PNE':
    print(f"⚠️ Warning: Expected PNE cycler, but detected {cycler_type}")

# Auto-detect capacity using pne_min_cap
capacity = pne_min_cap(folder_path, ini_crate)
print(f"📊 Auto-detected capacity: {capacity} mAh")

# Verify with folder name (should contain 4500mAh)
if '4500mAh' in FOLDER_NAME:
    expected_capacity = 4500
    print(f"✅ Capacity matches folder name: {expected_capacity} mAh")
else:
    print(f"ℹ️ Using auto-detected capacity: {capacity} mAh")

## 4. Load Cycle Data

In [ ]:
# Load cycle data
print("📥 Loading cycle data...")

try:
    cycle_df = pne_cycle_data(folder_path, capacity, ini_crate, chkir, chkir2, mkdcir)
    
    print(f"\n✅ Cycle data loaded successfully")
    print(f"📊 Total cycles: {len(cycle_df)}")
    print(f"📈 Cycle range: {cycle_df.index.min()} - {cycle_df.index.max()}")
    print(f"\n📋 DataFrame columns: {list(cycle_df.columns)}")
    print(f"\n📊 First 5 cycles:")
    print(cycle_df.head())
    
except Exception as e:
    print(f"❌ Error loading cycle data: {str(e)}")
    raise

## 5. Visualize Cycle Data (6-Panel Graph)

In [ ]:
# Create 6-panel visualization
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
fig.suptitle(f'PNE Cycle Analysis: {FOLDER_NAME}', fontsize=16, fontweight='bold')

cycles = cycle_df.index

# Panel 1: Discharge Capacity
axes[0, 0].plot(cycles, cycle_df['Dchg'], 'b-', linewidth=1.5)
axes[0, 0].set_title('Discharge Capacity vs Cycle', fontweight='bold')
axes[0, 0].set_xlabel('Cycle Number')
axes[0, 0].set_ylabel('Discharge Capacity (mAh)')
axes[0, 0].grid(True, alpha=0.3)

# Panel 2: Charge Capacity
axes[0, 1].plot(cycles, cycle_df['Chg'], 'r-', linewidth=1.5)
axes[0, 1].set_title('Charge Capacity vs Cycle', fontweight='bold')
axes[0, 1].set_xlabel('Cycle Number')
axes[0, 1].set_ylabel('Charge Capacity (mAh)')
axes[0, 1].grid(True, alpha=0.3)

# Panel 3: Efficiency
axes[1, 0].plot(cycles, cycle_df['Eff'], 'g-', linewidth=1.5)
axes[1, 0].set_title('Coulombic Efficiency vs Cycle', fontweight='bold')
axes[1, 0].set_xlabel('Cycle Number')
axes[1, 0].set_ylabel('Efficiency (%)')
axes[1, 0].set_ylim([95, 105])
axes[1, 0].grid(True, alpha=0.3)

# Panel 4: Temperature
if 'Temp' in cycle_df.columns:
    axes[1, 1].plot(cycles, cycle_df['Temp'], 'm-', linewidth=1.5)
    axes[1, 1].set_title('Temperature vs Cycle', fontweight='bold')
    axes[1, 1].set_xlabel('Cycle Number')
    axes[1, 1].set_ylabel('Temperature (°C)')
    axes[1, 1].grid(True, alpha=0.3)

# Panel 5: DCIR
if 'dcir' in cycle_df.columns:
    axes[2, 0].plot(cycles, cycle_df['dcir'], 'c-', linewidth=1.5)
    axes[2, 0].set_title('DCIR vs Cycle', fontweight='bold')
    axes[2, 0].set_xlabel('Cycle Number')
    axes[2, 0].set_ylabel('DCIR (mΩ)')
    axes[2, 0].grid(True, alpha=0.3)

# Panel 6: Discharge vs Charge Capacity Comparison
axes[2, 1].plot(cycles, cycle_df['Dchg'], 'b-', linewidth=1.5, label='Discharge', alpha=0.7)
axes[2, 1].plot(cycles, cycle_df['Chg'], 'r-', linewidth=1.5, label='Charge', alpha=0.7)
axes[2, 1].set_title('Discharge vs Charge Capacity', fontweight='bold')
axes[2, 1].set_xlabel('Cycle Number')
axes[2, 1].set_ylabel('Capacity (mAh)')
axes[2, 1].legend()
axes[2, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ 6-panel visualization complete")

## 6. Statistical Analysis

In [ ]:
# Overall statistics
print("="*80)
print("STATISTICAL ANALYSIS")
print("="*80)

print(f"\n📊 Discharge Capacity Statistics:")
print(f"  • Initial (cycle {cycle_df.index.min()}): {cycle_df['Dchg'].iloc[0]:.2f} mAh")
print(f"  • Final (cycle {cycle_df.index.max()}): {cycle_df['Dchg'].iloc[-1]:.2f} mAh")
print(f"  • Mean: {cycle_df['Dchg'].mean():.2f} mAh")
print(f"  • Min: {cycle_df['Dchg'].min():.2f} mAh (cycle {cycle_df['Dchg'].idxmin()})")
print(f"  • Max: {cycle_df['Dchg'].max():.2f} mAh (cycle {cycle_df['Dchg'].idxmax()})")
print(f"  • Std Dev: {cycle_df['Dchg'].std():.2f} mAh")

# Capacity retention
initial_capacity = cycle_df['Dchg'].iloc[0]
final_capacity = cycle_df['Dchg'].iloc[-1]
capacity_retention = (final_capacity / initial_capacity) * 100
print(f"\n📉 Capacity Retention: {capacity_retention:.2f}%")

print(f"\n📊 Charge Capacity Statistics:")
print(f"  • Initial: {cycle_df['Chg'].iloc[0]:.2f} mAh")
print(f"  • Final: {cycle_df['Chg'].iloc[-1]:.2f} mAh")
print(f"  • Mean: {cycle_df['Chg'].mean():.2f} mAh")
print(f"  • Min: {cycle_df['Chg'].min():.2f} mAh")
print(f"  • Max: {cycle_df['Chg'].max():.2f} mAh")

print(f"\n📊 Efficiency Statistics:")
print(f"  • Mean: {cycle_df['Eff'].mean():.3f}%")
print(f"  • Min: {cycle_df['Eff'].min():.3f}%")
print(f"  • Max: {cycle_df['Eff'].max():.3f}%")
print(f"  • Std Dev: {cycle_df['Eff'].std():.3f}%")

if 'Temp' in cycle_df.columns:
    print(f"\n🌡️ Temperature Statistics:")
    print(f"  • Mean: {cycle_df['Temp'].mean():.2f}°C")
    print(f"  • Min: {cycle_df['Temp'].min():.2f}°C")
    print(f"  • Max: {cycle_df['Temp'].max():.2f}°C")
    print(f"  • Std Dev: {cycle_df['Temp'].std():.2f}°C")

if 'dcir' in cycle_df.columns:
    dcir_valid = cycle_df['dcir'].dropna()
    if len(dcir_valid) > 0:
        print(f"\n⚡ DCIR Statistics:")
        print(f"  • Mean: {dcir_valid.mean():.3f} mΩ")
        print(f"  • Min: {dcir_valid.min():.3f} mΩ")
        print(f"  • Max: {dcir_valid.max():.3f} mΩ")
        print(f"  • Std Dev: {dcir_valid.std():.3f} mΩ")

## 7. Profile Analysis at Key Cycles

In [ ]:
# Select key cycles for profile analysis
max_cycle = cycle_df.index.max()
target_cycles = [
    2,                              # Early cycle
    min(10, max_cycle),             # 10th cycle or max
    min(50, max_cycle)              # 50th cycle or max
]

# Filter to only existing cycles
target_cycles = [c for c in target_cycles if c in cycle_df.index]

print(f"📊 Analyzing profiles for cycles: {target_cycles}")

# Store profile data
profiles_chg = {}
profiles_dchg = {}

for cycle_num in target_cycles:
    print(f"\n🔄 Loading profile for cycle {cycle_num}...")
    
    try:
        # Load charge profile
        chg_profile = pne_chg_Profile_data(
            folder_path, 
            cycle_num, 
            capacity, 
            cutoff, 
            ini_crate, 
            smoothdegree
        )
        profiles_chg[cycle_num] = chg_profile
        print(f"  ✅ Charge profile loaded: {len(chg_profile)} data points")
        
        # Load discharge profile
        dchg_profile = pne_dchg_Profile_data(
            folder_path, 
            cycle_num, 
            capacity, 
            cutoff, 
            ini_crate, 
            smoothdegree
        )
        profiles_dchg[cycle_num] = dchg_profile
        print(f"  ✅ Discharge profile loaded: {len(dchg_profile)} data points")
        
    except Exception as e:
        print(f"  ❌ Error loading profile for cycle {cycle_num}: {str(e)}")

print(f"\n✅ Profile analysis complete for {len(profiles_chg)} cycles")

## 8. Visualize Charge Profiles

In [ ]:
if profiles_chg:
    fig, axes = plt.subplots(2, 1, figsize=(12, 10))
    fig.suptitle(f'Charge Profiles: {FOLDER_NAME}', fontsize=14, fontweight='bold')
    
    colors = ['blue', 'green', 'red', 'purple', 'orange']
    
    for idx, (cycle_num, profile) in enumerate(profiles_chg.items()):
        color = colors[idx % len(colors)]
        
        # Voltage vs Capacity
        axes[0].plot(profile['Cap'], profile['Vol'], 
                    color=color, linewidth=2, label=f'Cycle {cycle_num}', alpha=0.7)
        
        # dQ/dV vs Voltage
        if 'dQdV' in profile.columns:
            axes[1].plot(profile['Vol'], profile['dQdV'], 
                        color=color, linewidth=2, label=f'Cycle {cycle_num}', alpha=0.7)
    
    # Configure voltage vs capacity plot
    axes[0].set_xlabel('Capacity (mAh)', fontweight='bold')
    axes[0].set_ylabel('Voltage (V)', fontweight='bold')
    axes[0].set_title('Charge: Voltage vs Capacity', fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Configure dQ/dV plot
    axes[1].set_xlabel('Voltage (V)', fontweight='bold')
    axes[1].set_ylabel('dQ/dV (mAh/V)', fontweight='bold')
    axes[1].set_title('Charge: dQ/dV Analysis', fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Charge profile visualization complete")
else:
    print("⚠️ No charge profiles available for visualization")

## 9. Visualize Discharge Profiles

In [ ]:
if profiles_dchg:
    fig, axes = plt.subplots(2, 1, figsize=(12, 10))
    fig.suptitle(f'Discharge Profiles: {FOLDER_NAME}', fontsize=14, fontweight='bold')
    
    colors = ['blue', 'green', 'red', 'purple', 'orange']
    
    for idx, (cycle_num, profile) in enumerate(profiles_dchg.items()):
        color = colors[idx % len(colors)]
        
        # Voltage vs Capacity
        axes[0].plot(profile['Cap'], profile['Vol'], 
                    color=color, linewidth=2, label=f'Cycle {cycle_num}', alpha=0.7)
        
        # dQ/dV vs Voltage
        if 'dQdV' in profile.columns:
            axes[1].plot(profile['Vol'], profile['dQdV'], 
                        color=color, linewidth=2, label=f'Cycle {cycle_num}', alpha=0.7)
    
    # Configure voltage vs capacity plot
    axes[0].set_xlabel('Capacity (mAh)', fontweight='bold')
    axes[0].set_ylabel('Voltage (V)', fontweight='bold')
    axes[0].set_title('Discharge: Voltage vs Capacity', fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Configure dQ/dV plot
    axes[1].set_xlabel('Voltage (V)', fontweight='bold')
    axes[1].set_ylabel('dQ/dV (mAh/V)', fontweight='bold')
    axes[1].set_title('Discharge: dQ/dV Analysis', fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Discharge profile visualization complete")
else:
    print("⚠️ No discharge profiles available for visualization")

## 10. Profile Comparison: Early vs Late Cycles

In [ ]:
if len(profiles_dchg) >= 2:
    # Compare first and last available cycles
    cycle_nums = sorted(profiles_dchg.keys())
    early_cycle = cycle_nums[0]
    late_cycle = cycle_nums[-1]
    
    early_profile = profiles_dchg[early_cycle]
    late_profile = profiles_dchg[late_cycle]
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    fig.suptitle(f'Discharge Profile Comparison: Cycle {early_cycle} vs {late_cycle}', 
                 fontsize=14, fontweight='bold')
    
    # Voltage vs Capacity comparison
    axes[0].plot(early_profile['Cap'], early_profile['Vol'], 
                'b-', linewidth=2, label=f'Cycle {early_cycle}', alpha=0.7)
    axes[0].plot(late_profile['Cap'], late_profile['Vol'], 
                'r-', linewidth=2, label=f'Cycle {late_cycle}', alpha=0.7)
    axes[0].set_xlabel('Capacity (mAh)', fontweight='bold')
    axes[0].set_ylabel('Voltage (V)', fontweight='bold')
    axes[0].set_title('Voltage vs Capacity', fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # dQ/dV comparison
    if 'dQdV' in early_profile.columns and 'dQdV' in late_profile.columns:
        axes[1].plot(early_profile['Vol'], early_profile['dQdV'], 
                    'b-', linewidth=2, label=f'Cycle {early_cycle}', alpha=0.7)
        axes[1].plot(late_profile['Vol'], late_profile['dQdV'], 
                    'r-', linewidth=2, label=f'Cycle {late_cycle}', alpha=0.7)
        axes[1].set_xlabel('Voltage (V)', fontweight='bold')
        axes[1].set_ylabel('dQ/dV (mAh/V)', fontweight='bold')
        axes[1].set_title('dQ/dV Analysis', fontweight='bold')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Calculate capacity difference
    early_cap = cycle_df.loc[early_cycle, 'Dchg']
    late_cap = cycle_df.loc[late_cycle, 'Dchg']
    cap_loss = early_cap - late_cap
    cap_loss_pct = (cap_loss / early_cap) * 100
    
    print(f"\n📊 Profile Comparison Summary:")
    print(f"  • Early cycle ({early_cycle}): {early_cap:.2f} mAh")
    print(f"  • Late cycle ({late_cycle}): {late_cap:.2f} mAh")
    print(f"  • Capacity loss: {cap_loss:.2f} mAh ({cap_loss_pct:.2f}%)")
    print("✅ Profile comparison complete")
else:
    print("⚠️ Need at least 2 cycles for comparison")

## 11. Export Results to Excel

In [ ]:
# Create export filename with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
export_filename = f"PNE_Single_{FOLDER_NAME}_{timestamp}.xlsx"
export_path = os.path.join(os.path.dirname(folder_path), export_filename)

print(f"💾 Exporting results to Excel...")
print(f"📁 Export path: {export_path}")

try:
    with pd.ExcelWriter(export_path, engine='openpyxl') as writer:
        # Export cycle data
        cycle_df.to_excel(writer, sheet_name='Cycle_Data')
        print("  ✅ Cycle data exported")
        
        # Export charge profiles
        for cycle_num, profile in profiles_chg.items():
            sheet_name = f'Chg_Profile_C{cycle_num}'
            profile.to_excel(writer, sheet_name=sheet_name, index=False)
            print(f"  ✅ Charge profile (cycle {cycle_num}) exported")
        
        # Export discharge profiles
        for cycle_num, profile in profiles_dchg.items():
            sheet_name = f'Dchg_Profile_C{cycle_num}'
            profile.to_excel(writer, sheet_name=sheet_name, index=False)
            print(f"  ✅ Discharge profile (cycle {cycle_num}) exported")
        
        # Export summary statistics
        summary_data = {
            'Metric': [
                'Total Cycles',
                'Cycle Range',
                'Initial Discharge Capacity (mAh)',
                'Final Discharge Capacity (mAh)',
                'Capacity Retention (%)',
                'Mean Discharge Capacity (mAh)',
                'Mean Charge Capacity (mAh)',
                'Mean Efficiency (%)',
                'Auto-detected Capacity (mAh)'
            ],
            'Value': [
                len(cycle_df),
                f"{cycle_df.index.min()} - {cycle_df.index.max()}",
                f"{cycle_df['Dchg'].iloc[0]:.2f}",
                f"{cycle_df['Dchg'].iloc[-1]:.2f}",
                f"{capacity_retention:.2f}",
                f"{cycle_df['Dchg'].mean():.2f}",
                f"{cycle_df['Chg'].mean():.2f}",
                f"{cycle_df['Eff'].mean():.3f}",
                f"{capacity}"
            ]
        }
        
        if 'Temp' in cycle_df.columns:
            summary_data['Metric'].append('Mean Temperature (°C)')
            summary_data['Value'].append(f"{cycle_df['Temp'].mean():.2f}")
        
        if 'dcir' in cycle_df.columns:
            dcir_valid = cycle_df['dcir'].dropna()
            if len(dcir_valid) > 0:
                summary_data['Metric'].append('Mean DCIR (mΩ)')
                summary_data['Value'].append(f"{dcir_valid.mean():.3f}")
        
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name='Summary', index=False)
        print("  ✅ Summary statistics exported")
    
    print(f"\n✅ Export complete: {export_filename}")
    
except Exception as e:
    print(f"❌ Error during export: {str(e)}")
    raise

## 12. Analysis Summary

In [ ]:
print("="*80)
print("ANALYSIS SUMMARY")
print("="*80)
print(f"\n📁 Data Folder: {FOLDER_NAME}")
print(f"📊 Cycler Type: PNE")
print(f"⚙️ Auto-detected Capacity: {capacity} mAh")
print(f"📈 Total Cycles Analyzed: {len(cycle_df)}")
print(f"🔢 Cycle Range: {cycle_df.index.min()} - {cycle_df.index.max()}")
print(f"\n📉 Capacity Retention: {capacity_retention:.2f}%")
print(f"📊 Mean Discharge Capacity: {cycle_df['Dchg'].mean():.2f} mAh")
print(f"📊 Mean Efficiency: {cycle_df['Eff'].mean():.3f}%")

if 'Temp' in cycle_df.columns:
    print(f"🌡️ Mean Temperature: {cycle_df['Temp'].mean():.2f}°C")

if 'dcir' in cycle_df.columns:
    dcir_valid = cycle_df['dcir'].dropna()
    if len(dcir_valid) > 0:
        print(f"⚡ Mean DCIR: {dcir_valid.mean():.3f} mΩ")

print(f"\n📂 Profiles Analyzed: {len(profiles_chg)} charge + {len(profiles_dchg)} discharge")
print(f"💾 Results Exported: {export_filename}")
print("\n" + "="*80)
print("✅ ANALYSIS COMPLETE")
print("="*80)